<a href="https://colab.research.google.com/github/AjitNikam1/ML_Automation/blob/master/Task1_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://shwetkm.github.io/upxlogo.png"></img>


# Live Project - Productionize Machine Learning models - Task 1(sample)

# New Section

# New Section

In [1]:
#Import basic packages

import warnings
warnings.filterwarnings("ignore")
import time
import pandas as pd        
import enum
          
import pickle
import sys

import plotly
from plotly.data import iris
import sklearn
from sklearn import datasets
from sklearn.model_selection import train_test_split   #splitting data
from pylab import rcParams
from sklearn.linear_model import LinearRegression         #linear regression
from sklearn.metrics.regression import mean_squared_error #error metrics
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
##
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from prettytable import PrettyTable
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC

#Code to import python module from classifier_sc.py file uploaded to your Google Drive
!pip install pydrive                             # Package to use Google Drive API - not installed in Colab VM by default
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth                    # Other necessary packages
from oauth2client.client import GoogleCredentials
auth.authenticate_user()                         # Follow prompt in the authorization process
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
your_module = drive.CreateFile({"id": "1c4uny6X2nv0kGR12zAOym1CMvoyDfyu-"})   # "your_module_file_id" is the part after "id=" in the shareable link
your_module.GetContentFile("classifier_sc.py")          # Save the .py module file to Colab VM
import classifier_sc  


In [0]:
#Import dictionaries for setting basic parameters and model parameters
from classifier_sc import classifier_config_dict,basic_params_dict,gridsearch_params_dict,MlProblemEnum,LabledDataEnum,TextDataEnum


In [0]:
#Import Dataset
df_iris = iris()
# #Import Boston Housing dataset for testing Regression Problem
# boston = datasets.load_boston()
# # Convert the original array data into a dataframe and append the column names.
# df_bston = pd.DataFrame(boston.data, columns = boston.feature_names)
# # Add a new variable in the dataframe for the target ( or label) variable
# df_bston['House_Price'] = boston.target

In [4]:
#View columns and paste in classifier_sc.py file with relevant columns
df_iris.columns
# df_bston.columns
df_iris.head()
print(type(len(df_iris.index)))

<class 'int'>


In [0]:
# Create a class to perform base model operations
class BaseModelHelper:
    def __init__(self,base_param,base_AlgoConfigDict,base_gridsearch_params,DataFrame,base_MlProblemEnum,base_LabledDataEnum,base_TextDataEnum):
        self.base_param = base_param
        self.base_AlgoConfigDict = base_AlgoConfigDict
        
        self.base_gridsearch_params=base_gridsearch_params
        #Initialise DataFrame
        self.dataframe = DataFrame
        #Initialize X
        self.X = self.base_param['X']
        #Initialize y
        self.y = self.base_param['y']
        # self.y = OutputcolumnName 
        #set random seed
        self.random_state = self.base_param['seed']
        #Set test_size
        self.test_size = self.base_param['test_size']
        self.base_MlProblemEnum = base_MlProblemEnum
        self.base_LabledDataEnum = base_LabledDataEnum
        self.base_TextDataEnum = base_TextDataEnum
        self.DataFrameCount = len(DataFrame.index)
        print('Inside BaseModelHelper')
        #Initialise AutoML object
        # self.AutoMLObj = AutoML(self.dataframe,self.base_MlProblemEnum,self.base_LabledDataEnum,self.base_TextDataEnum)
        #set base model params
        self.base_model = self.GetMlAlgoDictionary()
        self.base_gridsearch_params=self.base_gridsearch_params

        
    #Function to Get MlAlgoDictionary on the basis of Dataset Charactristics
    #Refered Scikit learn Algo selection Cheat Sheet - https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html 
    def GetMlAlgoDictionary(self):
    # Initialise ML Algo Dictinary
      MlAlgoDict = dict()
    # Check Problem type Classifiaton
      if self.base_MlProblemEnum is MlProblemEnum.Classification:
        #Check if have labled data or not 
        if self.base_LabledDataEnum is LabledDataEnum.Yes:
          #Check if Dataset samples is smaller than 100K
          if  self.DataFrameCount < 100000:
            MlAlgoDict.update({'LinearSVC': self.base_AlgoConfigDict['LinearSVC']})
            if self.base_TextDataEnum is TextDataEnum.Yes:
              MlAlgoDict.update({'KNN':self.base_AlgoConfigDict['KNN'],'SVM': self.base_AlgoConfigDict['SVM'],'RandomForestClassifier':self.base_AlgoConfigDict['RandomForestClassifier']})
              
      return MlAlgoDict 
     
       #Function to standardize columns
    def normalize_columns(self):
        
        # X=df_iris[self.X]
        X=self.dataframe[self.X]
        
        #Scale the values
        scaler = StandardScaler()
        scaler.fit(X)

        # Scale and center the data
        fdf_normalized = scaler.transform(X)

        # Create a pandas DataFrame
        fdf_normalized = pd.DataFrame(data=fdf_normalized, index=X.index, columns=X.columns)
        return fdf_normalized
    
    #Function to perform train test split
    def train_test_split_base(self,X_norm):
        self.X=X_norm
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X,self.dataframe[self.y],random_state=self.random_state,test_size=self.test_size)
    
    #GridsearchCV parameters based on the model
          
    def grid_params(self,model):
        #print(model)
        switcher={
            'RandomForestClassifier':self.base_gridsearch_params['RandomForestClassifier'],
            'DecisionTreeClassifier':self.base_gridsearch_params['DecisionTreeClassifier'],
            'LogisticRegression':self.base_gridsearch_params['LogisticRegression'],
            'SVM':self.base_gridsearch_params['SVM'],
            'KNN':self.base_gridsearch_params['KNN'],
            'LinearSVC':self.base_gridsearch_params['LinearSVC']
        }
        return switcher.get(model, "Invalid model")
        
    
    #Building model
    def model_build(self):
        
        X_norm = self.normalize_columns()
        b.train_test_split_base(X_norm)
        t = PrettyTable(['Name','Train Accuracy', 'Test Accuracy', 'Parameters'])
        t.format=False
        for key in self.base_model:
            model=self.base_model[key]
            model.fit(self.X_train,self.y_train)
            y_pred_test = model.predict(self.X_test)
           
                      
            #gridsearch cv
            random_grid = self.grid_params(str(key))
            
            rf_gs = GridSearchCV(model, random_grid, cv = 3, n_jobs=-1, verbose=2)

            rf_gs.fit(self.X_train,self.y_train)
            y_pred_test = rf_gs.predict(self.X_test)
            
            train_accuracy=accuracy_score(self.y_train, rf_gs.predict(self.X_train))*100
            test_accuracy=accuracy_score(self.y_test, y_pred_test)*100
           # print("Accuracy of GridSearchCV: {}%".format(accuracy_score(self.y_test, y_pred_test)*100))
            t.add_row([key,train_accuracy,test_accuracy,rf_gs.best_params_])
            #print(t)
        return t 

In [6]:
# Calling BaseModelHelper with basic_params_dict and classifier_config_dict from classifier_sc.py
 
b=BaseModelHelper(basic_params_dict,classifier_config_dict,gridsearch_params_dict,df_iris,MlProblemEnum.Classification,LabledDataEnum.Yes,TextDataEnum.Yes)
# Model Buildingfrom sklearn.svm import SVC 

t= b.model_build()
print(t)
Html_file= open("output.html","w")
Html_file.write(t.get_html_string())
Html_file.close()

Inside BaseModelHelper
Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    2.1s finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning:

The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    1.8s finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning:

The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.



Fitting 3 folds for each of 40 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 111 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    2.3s finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning:

The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.



Fitting 3 folds for each of 576 candidates, totalling 1728 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 116 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done 1084 tasks      | elapsed:   23.2s


+------------------------+-------------------+-------------------+---------------------------------------------------------------------------------------------------------------+
|          Name          |   Train Accuracy  |   Test Accuracy   |                                                   Parameters                                                  |
+------------------------+-------------------+-------------------+---------------------------------------------------------------------------------------------------------------+
|       LinearSVC        | 96.19047619047619 | 95.55555555555556 |             {'class_weight': 'balanced', 'dual': False, 'loss': 'squared_hinge', 'penalty': 'l2'}             |
|          KNN           |       100.0       | 93.33333333333333 |             {'algorithm': 'auto', 'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}             |
|          SVM           | 98.09523809523809 | 95.55555555555556 |                                    {'C

[Parallel(n_jobs=-1)]: Done 1728 out of 1728 | elapsed:   35.5s finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning:

The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.



In [0]:
testdict = dict()
testdict.update({'Key1':'Value1','Key2':'Value2'})
testdict

{'Key1': 'Value1', 'Key2': 'Value2'}